In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup, SoupStrainer
import time
from selenium.webdriver.common.keys import Keys
import os
import pandas as pd

In [3]:
#### This Script takes around 20 minutes to run to download all match data from 2017-current season

# Current Season
current_season = int(input('What is the current season?'))

# Load the HTML From Premier League
url = r'https://www.premierleague.com/tables?co=1&se=489&ha=-1'

# Disguise our User Agent

# HEADERS={'User-Agent':'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

driver = webdriver.Chrome()

r = driver.get(url) # proxies=proxies
time.sleep(4)

# Accept Cookies on Website

xpath_accept_cookies = r'//*[@id="onetrust-accept-btn-handler"]'
driver.find_element("xpath", xpath_accept_cookies).click()
time.sleep(2)

# Close the ads page if there are ads on the website

try:
    xpath_close_advert = r'//*[@id="advertClose"]'
    driver.find_element("xpath", xpath_close_advert).click()
    time.sleep(2)
except:
    print('No ads')



final_regular_table = pd.DataFrame()


# starts from 2022-2023 season (i=2) to 2017-2018 season (i=7)
for i in range (2,8):

    if not os.getcwd().endswith('Football Forecasting Version 2'):
        os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
    
    current_folder = os.getcwd()

    relative_folder = fr'{-i + current_season + 2}_{-i + current_season + 3}_Matchweek'   

    full_path = os.path.join(current_folder, relative_folder)

    isExist = os.path.exists(full_path)

    if not isExist:
        os.makedirs(full_path)
    
    
    xpath_filter_by_season = r'//*[@id="mainContent"]/div[2]/div[1]/div[1]/section/div[2]'
    driver.find_element("xpath", xpath_filter_by_season).click()
    time.sleep(4)

    xpath_season_i = fr'//*[@id="mainContent"]/div[2]/div[1]/div[1]/section/div[2]/ul/li[{i}]'
    driver.find_element("xpath", xpath_season_i).click()
    time.sleep(4)

    # starts from matchweek 1 (j=2) to matchweek 38 (j=39)  --> note: for 2022-2023 season, due to matchweek 7 cancellation max: j=38

    for j in range(2,40):
        
        try:
            # Filters by Matchweek

            xpath_filter_by_matchweek = r'//*[@id="mainContent"]/div[2]/div[1]/div[1]/section/div[3]/div[2]'
            driver.find_element("xpath", xpath_filter_by_matchweek).click()
            time.sleep(2)

            xpath_matchweek_i = fr'//*[@id="mainContent"]/div[2]/div[1]/div[1]/section/div[3]/ul/li[{j}]'
            driver.find_element("xpath", xpath_matchweek_i).click()
            time.sleep(3)

            # Creates the dataframe

            source=driver.page_source
            soup = BeautifulSoup(source,'html.parser') 
            table_soup = soup.find('table')

            header_data = []
            table_data = [] 
            #print(table_soup)
            row = table_soup.find('tr')
            header_data = [e.text.strip() for e in row.find_all('th')]

            for row in table_soup.find_all('tr'):
                row_text = [e.text.strip() for e in row.find_all('td')]
                table_data.append(row_text)

            regular_table = pd.DataFrame(table_data[1:], columns = header_data)

            # Cleaning the dataframes 

            regular_table ['Match Date'] = regular_table [1::2] ['More'].str.split('-').str[1]
            regular_table ['Match Date'] = regular_table [1::2] ['Match Date'].str.split(' ').str[2:5]
            a = []
            for x in regular_table ['Match Date']:
                try:
                    a.append(' '.join(x))
                except:
                    continue

            regular_table = regular_table [::2]
            regular_table = regular_table.drop(columns = ['More'])
            regular_table ['Match Date'] = a
            regular_table['Match Date'] = pd.to_datetime(regular_table ['Match Date'])
            regular_table.columns = ['Position', 'Club','Pl','W','D','L','GF','GA','GD','Pts','Form','Last Match Date']
            regular_table['Position'] = regular_table['Position'].str[:3].astype(int)
            regular_table['Code'] = regular_table['Club'].str[-3:]
            regular_table['Club'] = regular_table['Club'].str[:-3].str.strip()  
            regular_table['Pl'] = regular_table['Pl'].astype(int)
            regular_table['W'] = regular_table['W'].astype(int)
            regular_table['D'] = regular_table['D'].astype(int)
            regular_table['L'] = regular_table['L'].astype(int)
            regular_table['GF'] = regular_table['GF'].astype(int)
            regular_table['GA'] = regular_table['GA'].astype(int)
            regular_table['GD'] = regular_table['GD'].astype(int)
            regular_table['Pts'] = regular_table['Pts'].astype(int)
            regular_table['Form'] = regular_table['Form'].str.replace(r'\w{2,100}','',regex = True)
            regular_table['Form'] = regular_table['Form'].str.replace(r"\s+",'',regex = True)
            regular_table['Form'] = regular_table['Form'].str.replace(r'[^a-zA-Z]','',regex = True)
            regular_table['Form']= regular_table['Form'].apply(lambda x: (x.count('W')*3 + x.count('D'))/len(x)) 
            regular_table['Season'] = -i + current_season + 2
            regular_table.to_csv(fr'{full_path}\{-i + current_season + 2}_{-i + current_season + 3}_{j-1}.csv')

        except:
            continue



No ads


In [3]:
## Script for this matchweek

current_season = int(input('What is the current season?'))
current_matchweek = int(input('What is the current matchweek?'))

if not os.getcwd().endswith('Football Forecasting Version 2'):
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
    
current_folder = os.getcwd()

relative_folder = fr'{current_season}_{current_season + 1}_Matchweek'   

full_path = os.path.join(current_folder, relative_folder)

isExist = os.path.exists(full_path)

if not isExist:
    os.makedirs(full_path)

table = pd.read_html(r'https://www.premierleague.com/tables?co=1&se=489&ha=-1')



regular_table = pd.DataFrame(table[0])
regular_table = regular_table.drop(columns = ['Next','Unnamed: 12'])
regular_table ['Match Date'] = regular_table [1::2] ['Club'].str.split('-').str[1]
regular_table ['Match Date'] = regular_table [1::2] ['Match Date'].str.split(' ').str[2:5]
a = []
for x in regular_table ['Match Date'][1::2]:
    try:
        a.append(' '.join(x))
    except:
        continue

regular_table = regular_table [::2]
regular_table ['Match Date'] = a
regular_table['Match Date'] = pd.to_datetime(regular_table ['Match Date'])
regular_table.columns = ['Position', 'Club','Pl','W','D','L','GF','GA','GD','Pts','Form','Last Match Date']
regular_table['Position'] = regular_table['Position'].str[:3].astype(int)
regular_table['Code'] = regular_table['Club'].str[-3:]
regular_table['Club'] = regular_table['Club'].str[:-3].str.strip()
regular_table['Pl'] = regular_table['Pl'].astype(int)
regular_table['W'] = regular_table['W'].astype(int)
regular_table['D'] = regular_table['D'].astype(int)
regular_table['L'] = regular_table['L'].astype(int)
regular_table['GF'] = regular_table['GF'].astype(int)
regular_table['GA'] = regular_table['GA'].astype(int)
regular_table['GD'] = regular_table['GD'].astype(int)
regular_table['Pts'] = regular_table['Pts'].astype(int)
regular_table['Form'] = regular_table['Form'].str.replace(r'\w{2,100}','',regex = True)
regular_table['Form'] = regular_table['Form'].str.replace(r"\s+",'',regex = True)
regular_table['Form'] = regular_table['Form'].str.replace(r'[^a-zA-Z]','',regex = True)
regular_table['Form']= regular_table['Form'].apply(lambda x: (x.count('W')*3 + x.count('D'))/len(x)) 
regular_table['Season'] = current_season 

regular_table.to_csv(fr'{full_path}\{current_season}_{current_season + 1}_{current_matchweek -1}.csv')


In [ ]:
display(regular_table)

,Position,Club,Pl,W,D,L,GF,GA,GD,Pts,Form,Last Match Date,Code,Season
0,1,Arsenal,26,20,3,3,59,25,34,63,2.4,2023-03-04,ARS,2022
2,2,Manchester City,26,18,4,4,66,25,41,58,2.6,2023-03-04,MCI,2022
4,3,Manchester United,25,15,4,6,41,35,6,49,2.0,2023-03-05,MUN,2022
6,4,Tottenham Hotspur,26,14,3,9,46,36,10,45,1.8,2023-03-04,TOT,2022
8,5,Liverpool,25,12,6,7,47,28,19,42,2.6,2023-03-05,LIV,2022
10,6,Newcastle United,24,10,11,3,35,17,18,41,0.6,2023-03-04,NEW,2022
12,7,Fulham,26,11,6,9,38,34,4,39,1.6,2023-03-06,FUL,2022
14,8,Brighton and Hove Albion,23,11,5,7,43,29,14,38,1.6,2023-03-04,BHA,2022
16,9,Brentford,24,9,11,4,40,32,8,38,1.8,2023-03-06,BRE,2022
18,10,Chelsea,25,9,7,9,24,25,-1,34,1.0,2023-03-04,CHE,2022
